In [2]:
import pandas as pd

acs_data = pd.read_csv("census_data.csv")


required_columns = ['County', 'State', 'TotalPop', 'IncomePerCap', 'Poverty']
acs_data_filtered = acs_data[required_columns].copy()


# Create a new column 'ID' as a unique identifier for the county
acs_data_filtered['ID'] = range(len(acs_data_filtered))

# Group the data by 'County' and 'State' and aggregate the values using appropriate aggregation functions
aggregated_data = acs_data_filtered.groupby(['County', 'State']).agg({'TotalPop': 'sum', 'IncomePerCap': 'mean', 'Poverty': 'mean'})

# Reset the index of the aggregated DataFrame
County_Info = aggregated_data.reset_index()

# Identify the most populous county
most_populous_county = County_Info.loc[County_Info['TotalPop'].idxmax()]
print("Most populous county:")
print(most_populous_county[['County', 'State', 'TotalPop']])

# Identify the least populous county
least_populous_county = County_Info.loc[County_Info['TotalPop'].idxmin()]
print("Least populous county:")
print(least_populous_county[['County', 'State', 'TotalPop']])

# Rename the columns to match the desired column names
County_Info.rename(columns={'County': 'county','TotalPop': 'Population', 'IncomePerCap': 'PerCapitaIncome'}, inplace=True)

# Reset the 'ID' column to start from 1
County_Info['ID'] = County_Info.index + 1


county_inputs = ["Loudoun County, Virginia", "Washington County, Oregon", "Harlan County, Kentucky", "Malheur County, Oregon"]

# Initialize an empty DataFrame with the same columns as County_Info
county_details = pd.DataFrame(columns=County_Info.columns)

for input_county in county_inputs:
    county_name, state_name = input_county.split(", ")
    details = County_Info[(County_Info['county'] == county_name) & (County_Info['State'] == state_name)]
    county_details = pd.concat([county_details, details])

print(county_details)



Most populous county:
County      Los Angeles County
State               California
TotalPop              10105722
Name: 1757, dtype: object
Least populous county:
County      Loving County
State               Texas
TotalPop               74
Name: 1764, dtype: object
                 county     State Population  PerCapitaIncome    Poverty    ID
1759     Loudoun County  Virginia     374558     50391.015625   3.884375  1760
3046  Washington County    Oregon     572071     34970.817308  10.446154  3047
1234      Harlan County  Kentucky      27548     16010.363636  33.318182  1235
1822     Malheur County    Oregon      30421     17966.428571  24.414286  1823


In [3]:
covid_data = pd.read_csv("covid_county.csv")

# # Assuming the dataset is stored in a DataFrame called 'covid_data'
covid_data['ID'] = covid_data['county'] + '_' + covid_data['state']
covid_data['ID'] = pd.factorize(covid_data['ID'])[0] + 1

covid_data['date'] = pd.to_datetime(covid_data['date'], format='%Y-%m-%d')

covid_data['month'] = covid_data['date'].dt.to_period('M')


covid_data_monthly = covid_data.groupby(['ID', 'month']).agg({
    'cases': 'sum',
    'deaths': 'sum',
    'county': 'first'
}).reset_index()

covid_data_monthly['ID'] = range(1, len(covid_data_monthly) + 1)
covid_data_monthly = covid_data_monthly[['ID', 'month', 'cases', 'deaths', 'county']].copy()

covid_data_monthly.columns = ['ID', 'Month', 'cases', 'deaths', 'county']


selected_months = ['2020-08', '2021-01', '2021-02']
selected_county = 'Malheur'
covid_report = covid_data_monthly[(covid_data_monthly['county'] == selected_county) &
                              (covid_data_monthly['Month'].astype(str).isin(selected_months))]

print(covid_report)

          ID    Month  cases  deaths   county
25593  25594  2020-08  28163   459.0  Malheur
25598  25599  2021-01  96297  1627.0  Malheur
25599  25600  2021-02  65951  1137.0  Malheur


In [4]:
covid_summary = (covid_data_monthly.groupby("ID").agg({"cases": "sum", "deaths": "sum"}).reset_index())
covid_summary.rename(columns={"cases": "TotalCases", "deaths": "TotalDeaths"}, inplace=True)
covid_summary = pd.merge(covid_summary, County_Info, how="left", on="ID")
covid_summary["TotalCasesPer100K"] = round((covid_summary["TotalCases"]) / (covid_summary["Population"] / 100000), 2)
covid_summary["TotalDeathsPer100K"] = round((covid_summary["TotalDeaths"]) / (covid_summary["Population"] / 100000), 2)


counties = [('Washington County', 'Oregon'), ('Malheur County', 'Oregon'), ('Loudoun County', 'Virginia'), ('Harlan County', 'Kentucky')]

# Convert the list of tuples into a DataFrame for easier merging
county_state_df = pd.DataFrame(counties, columns=['county', 'State'])
filtered_df = pd.merge(covid_summary, county_state_df, on=['county', 'State'], how='inner')

# Extract the 'Poverty%' and 'TotalCasesPer100K' values for the specified counties
poverty_percentage = filtered_df[['county', 'State', 'Poverty']]
total_cases_per_100k = filtered_df[['county', 'State', 'TotalCasesPer100K']]

# Print the results
print("Poverty Percentage:")
print(poverty_percentage)
print("\nTotal Cases per 100K:")
print(total_cases_per_100k)

Poverty Percentage:
              county     State    Poverty
0      Harlan County  Kentucky  33.318182
1     Loudoun County  Virginia   3.884375
2     Malheur County    Oregon  24.414286
3  Washington County    Oregon  10.446154

Total Cases per 100K:
              county     State  TotalCasesPer100K
0      Harlan County  Kentucky         1671979.82
1     Loudoun County  Virginia            3086.84
2     Malheur County    Oregon          542792.81
3  Washington County    Oregon           61741.46


In [5]:
oregon_county = covid_summary[covid_summary['State'] == 'Oregon']
print(oregon_county)


#COVID total cases vs. % population in poverty

analysis_1a = oregon_county['TotalCases'].corr(oregon_county['Poverty'])



#COVID total deaths vs. % population in poverty

analysis_1b = oregon_county['TotalDeaths'].corr(oregon_county['Poverty'])
print(analysis_1b)


#COVID total cases vs. Per Capita Income level

analysis_1c = oregon_county['TotalCases'].corr(oregon_county['PerCapitaIncome'])
print(analysis_1c)

#COVID total deaths vs. Per Capita Income level

analysis_1d = oregon_county['TotalDeaths'].corr(oregon_county['PerCapitaIncome'])
print(analysis_1d)


#COVID total cases vs. % population in poverty

r2 = covid_summary['TotalCases'].corr(covid_summary['Poverty'])
print(r2)


#COVID total deaths vs. % population in poverty

r2 = covid_summary['TotalDeaths'].corr(covid_summary['Poverty'])
print(r2)


#COVID total cases vs. Per Capita Income level

r2 = covid_summary['TotalCases'].corr(covid_summary['PerCapitaIncome'])
print(r2)


#COVID total deaths vs. Per Capita Income level

r2 = covid_summary['TotalDeaths'].corr(covid_summary['PerCapitaIncome'])
print(r2)

        ID  TotalCases  TotalDeaths             county   State  Population  \
126    127      176630       7094.0       Baker County  Oregon     15980.0   
197    198      269042       2377.0      Benton County  Oregon     88249.0   
533    534       64549       2655.0   Clackamas County  Oregon    399962.0   
558    559      165400       8893.0     Clatsop County  Oregon     38021.0   
631    632        9177        720.0    Columbia County  Oregon     50207.0   
656    657       25645        265.0        Coos County  Oregon     62921.0   
697    698     1099390      19381.0       Crook County  Oregon     21717.0   
718    719      294898       2637.0       Curry County  Oregon     22377.0   
796    797         455         21.0   Deschutes County  Oregon    175321.0   
839    840      513374       8400.0     Douglas County  Oregon    107576.0   
1068  1069     1247029      27789.0     Gilliam County  Oregon      1910.0   
1115  1116       11712         93.0       Grant County  Oregon  